In [23]:
import pandas as pd
import os, glob
import seaborn as sns

import matplotlib.pyplot as plt

In [3]:
processed_path = "../data_processed/"
blasted_path = os.path.join(processed_path, "blasted")

In [5]:
train_files = glob.glob(blasted_path+"/blasted_train*")
train_files

['../data_processed/blasted/blasted_train_seqs_group_54.txt',
 '../data_processed/blasted/blasted_train_seqs_group_36.txt',
 '../data_processed/blasted/blasted_train_seqs_group_59.txt',
 '../data_processed/blasted/blasted_train_seqs_group_20.txt',
 '../data_processed/blasted/blasted_train_seqs_group_82.txt',
 '../data_processed/blasted/blasted_train_seqs_group_24.txt']

In [16]:
cols = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 
        'gapopen', 'sstart', 'send', 'evalue', 'staxids', 
        'sscinames', 'sblastnames', 'stitle']

eval_cutoff = 0.01

each `sseqid` correspond to a unique `stitle`, so `stitle` is a bit redundent.
But we can parse stitle to get a higher level info

In [10]:
df = pd.read_table(train_files[0], names=cols)
df.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,sstart,send,evalue,staxids,sscinames,sblastnames,stitle
0,8AQG1,ref|XM_027388357.2|,74.434,618,155,2,1044,430,1.630000e-65,NaN,NaN,NaN,PREDICTED: Cricetulus griseus neuropeptide FF ...
1,6AQTQ,ref|XM_003804830.3|,98.837,86,0,1,2149,2065,3.710000e-32,NaN,NaN,NaN,PREDICTED: Pan paniscus arginine vasopressin r...
2,AHPUB,ref|XM_014851462.1|,77.392,836,174,9,1387,561,1.140000e-131,NaN,NaN,NaN,PREDICTED: Equus asinus neuropeptide Y recepto...
3,8YT6U,ref|XM_003804830.3|,98.837,86,0,1,2149,2065,3.570000e-32,NaN,NaN,NaN,PREDICTED: Pan paniscus arginine vasopressin r...
4,JUCIL,ref|XM_017667393.1|,72.279,873,224,16,1769,906,2.810000e-63,NaN,NaN,NaN,PREDICTED: Manis javanica neuropeptide Y recep...


In [32]:
# for each query seqid, take the first one (because it is the best alignment result)
# remove record with evalue > cutoff (0.01), because it might not be reliable
df_unique = df.groupby('qseqid').first()
print("before filtering:", df_unique.shape)
df_unique = df_unique[df_unique.evalue<eval_cutoff]
print("after filtering:", df_unique.shape)

before filtering: (630, 12)
after filtering: (630, 12)


In [34]:
print(df_unique.stitle.unique())

['Human ORFeome Gateway entry vector pENTR223-BTF3, complete sequence'
 'PREDICTED: Hylobates moloch Sec23 homolog A, COPII coat complex component (SEC23A), transcript variant X2, mRNA'
 'Cloning vector PJAC98, complete sequence'
 'Cloning vector p172, complete sequence'
 'Synthetic construct Homo sapiens clone IMAGE:100061543, MGC:190020 general transcription factor IIE, polypeptide 1, alpha 56kDa (GTF2E1) mRNA, encodes complete protein'
 'Human ORFeome Gateway entry vector pENTR223-CSNK2A1, complete sequence'
 'Gateway binary vector pGWB788 DNA, complete sequence'
 'Human ORFeome Gateway entry vector pENTR223-ARF6, complete sequence'
 'Reverse genetics vector pC-N, complete sequence'
 'Cloning vector pTARBAC6, complete sequence'
 'Cloning vector pcDNA3.1_+, complete sequence'
 'Synthetic construct Homo sapiens clone ccsbBroadEn_14136 SAR1B gene, encodes complete protein'
 'Human ORFeome Gateway entry vector pENTR223-BTF3L4, complete sequence'
 'Expression vector pKK44, complete seque